# Iris Vector Database RAG Integration with Evidently

This notebook demonstrates how to use InterSystems IRIS as a vector database for RAG (Retrieval-Augmented Generation) with Evidently's LLM evaluation capabilities.

## Overview

We'll cover:
1. Setting up IRIS vector database with sample data
2. Using Evidently's RAG module with IRIS backend
3. Performing similarity searches and evaluations

## Prerequisites

- IRIS instance running locally (Docker recommended)
- Python packages: `intersystems-irispython`, `evidently[llm,iris]`, `sentence-transformers`

## Step 1: Install Required Packages

In [ ]:
# Install required packages
# ! pip install intersystems-irispython evidently[llm,iris] sentence-transformers

## Step 2: Setup IRIS Vector Database

First, we'll connect to IRIS and create a vector table with sample data.

In [ ]:
import iris
from sentence_transformers import SentenceTransformer

# IRIS connection parameters
username = 'SuperUser'
password = ''
hostname = 'localhost'
port = 1972
namespace = 'USER'

# Create connection string
connection_string = f"{hostname}:{port}/{namespace}"

# Connect to IRIS
connection = iris.connect(connection_string, username, password)
cursor = connection.cursor()

print(f"Connected to IRIS at {connection_string}")

In [ ]:
# Create vector table for our RAG data
table_name = "evidently_rag_demo"
table_definition = """(
    ID INT PRIMARY KEY,
    text VARCHAR(2000),
    vector_data VECTOR(FLOAT, 384),
    source VARCHAR(200)
)"""

# Drop table if exists and recreate
try:
    cursor.execute(f"DROP TABLE {table_name}")
except:
    pass

cursor.execute(f"CREATE TABLE {table_name} {table_definition}")
print(f"Created table: {table_name}")

In [ ]:
# Sample documents for our RAG system
sample_documents = [
    {
        "text": "Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data without being explicitly programmed.",
        "source": "ml_basics.txt"
    },
    {
        "text": "Deep learning uses neural networks with multiple layers to model and understand complex patterns in data. It's particularly effective for image recognition and natural language processing.",
        "source": "deep_learning.txt"
    },
    {
        "text": "Natural language processing (NLP) enables computers to understand, interpret, and generate human language. It's used in chatbots, translation, and sentiment analysis.",
        "source": "nlp_intro.txt"
    },
    {
        "text": "Vector databases store and search high-dimensional vectors efficiently. They're essential for semantic search, recommendation systems, and RAG applications.",
        "source": "vector_db.txt"
    },
    {
        "text": "Retrieval-Augmented Generation (RAG) combines information retrieval with language generation to provide more accurate and contextual responses.",
        "source": "rag_concept.txt"
    }
]

# Initialize sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Insert documents with embeddings
for i, doc in enumerate(sample_documents):
    # Generate embedding
    embedding = model.encode(doc['text']).tolist()
    
    # Insert into IRIS table
    cursor.execute(
        f"INSERT INTO {table_name} (ID, text, vector_data, source) VALUES (?, ?, ?, ?)",
        [i + 1, doc['text'], str(embedding), doc['source']]
    )

connection.commit()
print(f"Inserted {len(sample_documents)} documents into {table_name}")

## Step 3: Test Vector Search in IRIS

Let's test the vector similarity search directly in IRIS before using Evidently.

In [ ]:
# Test vector similarity search
query_text = "What is machine learning and how does it work?"
query_vector = model.encode(query_text).tolist()

# Perform similarity search
search_sql = f"""
SELECT TOP 3 ID, text, source
FROM {table_name}
ORDER BY VECTOR_DOT_PRODUCT(vector_data, ?) DESC
"""

cursor.execute(search_sql, [str(query_vector)])
results = cursor.fetchall()

print(f"Query: {query_text}")
print("\nMost relevant documents:")
for i, result in enumerate(results, 1):
    print(f"{i}. ID: {result[0]}, Source: {result[2]}")
    print(f"   Text: {result[1]}")
    print()

## Step 4: Using Evidently RAG with IRIS

Now let's use Evidently's RAG module with our IRIS vector database using the new flat parameter API.

In [ ]:
from evidently.llm.rag.iris_index import IrisDataCollectionProvider
from evidently.llm.rag.splitter import Chunk

# Create IRIS data collection provider with flat parameters
iris_provider = IrisDataCollectionProvider(
    # Connection parameters (all required except port)
    hostname=hostname,
    port=port,  # defaults to 1972 if not specified
    namespace=namespace,
    username=username,
    password=password,
    # Table configuration
    table_name=table_name,
    text_column="text",
    vector_column="vector_data",
    id_column="ID"
)

print("Created IRIS data collection provider with flat parameters")

In [ ]:
# Get the data collection and test it
data_collection = iris_provider.get_data_collection()

# Test queries
test_queries = [
    "What is machine learning?",
    "How do neural networks work?",
    "What is RAG and why is it useful?",
    "Tell me about vector databases"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    print("-" * 50)
    
    # Find relevant chunks
    relevant_chunks = data_collection.find_relevant_chunks(query, n_results=2)
    
    for i, chunk in enumerate(relevant_chunks, 1):
        print(f"{i}. {chunk}")
        print()

## Step 5: RAG Dataset Generation

Now let's use Evidently's RAG dataset generation capabilities with our IRIS backend.

In [ ]:
from evidently.llm.datagen.rag import RagDatasetGenerator
from evidently.llm.datagen.config import GenerationSpec

# Create RAG dataset generator using our IRIS provider
rag_generator = RagDatasetGenerator(
    data_collection=iris_provider,
    count=5,  # Generate 5 question-answer pairs
    model="gpt-4o-mini",
    provider="openai",
    query_spec=GenerationSpec(kind="question", complexity="medium"),
    response_spec=GenerationSpec(kind="answer", complexity="medium")
)

print("Created RAG dataset generator with IRIS backend")

In [ ]:
# Generate dataset (requires OpenAI API key)
import os

# Note: You'll need to set your OpenAI API key
# os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

dataset_result = rag_generator.generate()

print(f"Generated {len(dataset_result)} question-answer pairs")
for i, gen in dataset_result.iterrows():
    print(f"\n{i+1}. Question: {gen[0]}")
    print(f"   Answer: {gen[1]}")

print("RAG dataset generation ready (requires OpenAI API key)")

## Step 6: Cleanup and Best Practices

Let's clean up our resources and discuss best practices.

In [ ]:
# Cleanup: Close connections
if 'data_collection' in locals():
    data_collection.close()

if 'connection' in locals():
    connection.close()

print("Closed all database connections")

## Summary and Best Practices

### What We've Accomplished:

1. **IRIS Vector Database Setup**: Created a vector table and populated it with sample documents
2. **Evidently Integration**: Used `IrisDataCollectionProvider` with flat parameters to integrate IRIS with Evidently's RAG system
3. **Similarity Search**: Demonstrated vector similarity search using `VECTOR_DOT_PRODUCT`
4. **RAG Dataset Generation**: Set up RAG dataset generation with IRIS backend

### Key Features of the New API:

1. **Flat Parameters**: The provider accepts connection parameters directly instead of a connection object
2. **Required Parameters**: Most connection parameters are required (except port which defaults to 1972)
3. **Type Safety**: Port is now an integer type
4. **Connection Management**: `IrisConnection` handles cursor and connection management
5. **Simplified Usage**: No need to create connection objects manually

### Best Practices:

1. **Connection Management**: Always close database connections when done
2. **Error Handling**: Implement proper error handling for database operations
3. **Indexing**: Consider creating indexes on frequently queried columns
4. **Batch Operations**: Use batch inserts for large datasets
5. **Monitoring**: Monitor query performance and optimize as needed

### Next Steps:

1. Set up your OpenAI API key to test dataset generation
2. Experiment with different embedding models
3. Try different similarity search functions (cosine, etc.)
4. Implement batch document processing
5. Add monitoring and logging for production use